In [1]:
import sys
import os

# Get the path to src/ relative to this notebook
notebook_dir = os.getcwd()
src_path = os.path.abspath(os.path.join(notebook_dir, "../../src"))

# Add to Python path if not already there
if src_path not in sys.path:
    sys.path.insert(0, src_path)

from utils.chelsa import extract_param
from datasource.chelsa import s3, layer, sampling

# Generate CHELSA cube

In [ ]:
param_file = "prototype_param.yaml"
chelsa_month_param = extract_param.read_chelsa_month_param(param_file, param_path = "../../config")
chelsa_ref_period_param = extract_param.read_chelsa_clim_ref_period_param(param_file, param_path = "../../config")
chelsa_ref_month_param = extract_param.read_chelsa_clim_ref_month_param(param_file, param_path = "../../config")
chelsa_clim_period_param = extract_param.read_chelsa_clim_sim_period_param(param_file, param_path = "../../config")
chelsa_clim_month_param = extract_param.read_chelsa_clim_sim_month_param(param_file, param_path = "../../config")

In [ ]:
def da_layer_constructor(data_layer_func, param):
    static_param = list(param.values())[1:]
    data_arrays = []
    for var in param["var"]:
        data_arrays.append(data_layer_func(var, *static_param))
    return data_arrays

In [ ]:
test = da_layer_constructor(layer.chelsa_month_ts, chelsa_month_param)

In [ ]:
test

In [ ]:
import xarray

def regrid_interp(
    da: xr.DataArray,
    new_lats: xr.DataArray,
    new_lons: xr.DataArray,
    method: str = "linear"
) -> xr.DataArray:
    """
    Upscale or downscale by interpolating `da` onto (new_lats, new_lons).
    `method` can be "linear", "nearest", or "cubic".
    """
    # Ensure monotonic coords
    da = da.sortby(["lat", "lon"])
    return da.interp(lat=new_lats, lon=new_lons, method=method)

def reproject_array(data_array, upscaled_long, upscaled_lat, method="cubic"):
    

In [ ]:
import numpy as np

def compare_coord_ticks(coord_ticks):
    ref = coord_ticks[0]
    differences = []  
    for i, arr in enumerate(coord_ticks):
        if len(arr) != len(ref):
            differences.append((i, "different length"))
        elif not np.array_equal(arr, ref):
            differences.append((i, "same length, different values")) 
    return differences

In [ ]:
long_coordinates = [da[1].long.values for da in test]
lat_coordinates = [da[1].lat.values for da in test]

def group_by_size(coordinates):
    #Retrieve size of coordinates
    coordinates_sizes = [len(coordinate_array) for coordinate_array in coordinates]
    #Group coordinates that are similar in size together
    co_idx = [[idx for idx, el in enumerate(coordinates_sizes) if el==size] for size in list(set(coordinates_sizes))]
    return co_idx

def check_homogeneity(coordinates):
    if len(coordinates)==1:
        return True
    else:
        
group_by_size(lat_coordinates)

In [ ]:
def harmonize_data_arrays(data_arrays, 
                          harmonization="upscale", method="linear", fill_na=True):
    if harmonization=="upscale":
        long_coordinates = [da.long.values for da in data_arrays]
        latitude_coordinates = [da.lat.values for da in data_arrays]
        target_long = 
        target_lat

In [ ]:
class spatiotemporal_cube():
    def __init__(self):
        pass

    def da_layer_constructor(self, data_layer_func, param):
        static_param = list(param.values())[1:]
        data_arrays = []
        for var in param["var"]:
            data_arrays.append(data_layer_func(var, *static_param))
        return data_arrays

    def da_concat(self, data_arrays, dim_name, coordinates):
        combined_data_array = xr.concat(data_arrays, dim=dim_name)
        combined_data_array = combined_data_array.assign_coords({dim_name: coordinates})
        return combined_data_array
        
    def regrid_spatial_coordinates(self, data_array, target_lat, target_long,
                                   harmonization="upscale", method="linear", fill_nearest=True):
        if harmonization=="upscale":
            upscaled_data_array = data_array.interp(lat=target_lat, long=target_long, method=method)
            if fill_nearest:
                upscaled_filled_data_array = upscaled_data_array.ffill(dim="lat").bfill(dim="lat").ffill(dim="long").bfill(dim="long")
                return upscaled_filled_data_array
            else:
                return upscaled_data_array

In [ ]:
chelsa_ref_period_param = extract_param.read_chelsa_clim_ref_period_param(param_file, param_path = "../../config")
chelsa_ref_month_param = extract_param.read_chelsa_clim_ref_month_param(param_file, param_path = "../../config")
chelsa_clim_period_param = extract_param.read_chelsa_clim_sim_period_param(param_file, param_path = "../../config")
chelsa_clim_month_param = extract_param.read_chelsa_clim_sim_month_param(param_file, param_path = "../../config")

In [2]:
import xarray as xr
import yaml

class spatiotemporal_cube():
    def __init__(self):
        pass

    def da_layer_constructor(self, data_layer_func, param):
        static_param = list(param.values())[1:]
        data_arrays = []
        for var in param["var"]:
            data_arrays.append(data_layer_func(var, *static_param))
        return data_arrays

    def da_concat(self, data_arrays, dim_name, coordinates):
        combined_data_array = xr.concat(data_arrays, dim=dim_name)
        combined_data_array = combined_data_array.assign_coords({dim_name: coordinates})
        return combined_data_array
        
    def regrid_spatial_coordinates(self, data_array, target_lat, target_long,
                                   harmonization="upscale", method="linear", fill_nearest=True):
        if harmonization=="upscale":
            upscaled_data_array = data_array.interp(lat=target_lat, long=target_long, method=method)
            if fill_nearest:
                upscaled_filled_data_array = upscaled_data_array.ffill(dim="lat").bfill(dim="lat").ffill(dim="long").bfill(dim="long")
                return upscaled_filled_data_array
            else:
                return upscaled_data_array
                
class chelsa_cube(spatiotemporal_cube):
    def __init__(self):
        self.layers = ["chelsa_month", 
                       "chelsa_clim_ref_period",
                       "chelsa_clim_ref_month",
                       "chelsa_clim_sim_period",
                       "chelsa_clim_sim_month"]
        self.data = []

    #generate monthly data for a set of variables
    def generate_chelsa_month_layer(self, param_file, param_path = "../../config"):
        chelsa_month_param = extract_param.read_chelsa_month_param(param_file, param_path = param_path)
        var_names, data = zip(*self.da_layer_constructor(layer.chelsa_month_ts, chelsa_month_param)) #returns list of tuples (var_name, data)
        #Current assumption is that all variables will be requested
        #In future general harmonization functions will be implemented 
        #Select the coordinates from the second variable, clt is the only one that differs
        target_lat = data[1].lat.values
        target_long = data[1].long.values
        upscaled_clt = self.regrid_spatial_coordinates(data[0], target_lat, target_long)
        data = [upscaled_clt]+list(data[1:])
        return self.da_concat(data, "variable", list(var_names))

    def generate_chelsa_ref_period_layer(self, param_file, param_path="../../config"):
        chelsa_ref_period_param = extract_param.read_chelsa_clim_ref_period_param(param_file, param_path = param_path)
        var_names, data = zip(*self.da_layer_constructor(layer.chelsa_clim_ref_period, chelsa_ref_period_param))
        upscale_idx = [var_names.index(el) for el in ['clt_max', 'clt_mean', 'clt_min', 'clt_range'] if el in var_names]
        data = list(data)
        if upscale_idx:
            #Select the idx of data array that does not need to be upscaled
            #The difference between sets generates the set containing idx that are not in upscale idx
            target_idx = list(set(list(range(len(var_names))))-set(upscale_idx))[0]
            target_lat = data[target_idx].lat.values
            target_long = data[target_idx].long.values
            upscaled_da = [self.regrid_spatial_coordinates(data[idx], target_lat, target_long) for idx in upscale_idx]
            for idx, da in zip(upscale_idx, upscaled_da):
                data[idx] = da
        #return var_names, data
        return self.da_concat(data, "variable", list(var_names))

    def generate_chelsa_ref_month_layer(self, param_file, param_path="../../config"):
        chelsa_ref_month_param = extract_param.read_chelsa_clim_ref_month_param(param_file, param_path = param_path)
        var_names, data = zip(*self.da_layer_constructor(layer.chelsa_clim_ref_month, chelsa_ref_month_param))
        #Current assumption is that all variables will be requested
        #In future general harmonization functions will be implemented 
        #Select the coordinates from the second variable, clt is the only one that differs
        target_lat = data[1].lat.values
        target_long = data[1].long.values
        upscaled_clt = self.regrid_spatial_coordinates(data[0], target_lat, target_long)
        data = [upscaled_clt]+list(data[1:])
        return self.da_concat(data, "variable", list(var_names))

    def generate_chelsa_sim_period_layer(self, param_file, param_path="../../config"):
        chelsa_sim_period_param = extract_param.read_chelsa_clim_sim_period_param(param_file, param_path = param_path)
        var_names, data = zip(*self.da_layer_constructor(layer.chelsa_clim_sim_period, chelsa_sim_period_param))
        return self.da_concat(data, "variable", list(var_names))

    def generate_chelsa_sim_month_layer(self, param_file, param_path="../../config"):
        chelsa_sim_month_param = extract_param.read_chelsa_clim_sim_month_param(param_file, param_path = param_path)
        var_names, data = zip(*self.da_layer_constructor(layer.chelsa_clim_sim_month, chelsa_sim_month_param))
        return self.da_concat(data, "variable", list(var_names))

    def generate_chelsa_cube(self, param_file, param_path="../../config"):
        param_filepath = os.path.join(param_path, param_file)
        with open(param_filepath) as f:
            param_dict = yaml.safe_load(f)
        enabled_layers = [param_dict["layers"][layer] for layer in self.layers]
        functions = [self.generate_chelsa_month_layer, 
                     self.generate_chelsa_ref_period_layer,
                     self.generate_chelsa_ref_month_layer,
                     self.generate_chelsa_sim_period_layer,
                     self.generate_chelsa_sim_month_layer]
        enabled_layers_names = [name for name, call in zip(self.layers, enabled_layers) if call]
        data = [func(param_file, param_path) for func, call in zip(functions, enabled_layers) if call]
        return dict(zip(enabled_layers_names, data))

In [3]:
test_chelsa_cube = chelsa_cube()

In [109]:
test_ds = test_chelsa_cube.generate_chelsa_cube("prototype_param.yaml")

-----Retrieving monthly CHELSA data for variable 'clt'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.16item/s]


-----Retrieving monthly CHELSA data for variable 'cmi'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.13item/s]


-----Retrieving monthly CHELSA data for variable 'hurs'-----


...

Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:17<00:00,  5.07item/s]


-----Retrieving Simulation (monthly) CHELSA data for variable 'tasmax'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:17<00:00,  5.24item/s]


-----Retrieving Simulation (monthly) CHELSA data for variable 'tasmin'-----


Processing: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:16<00:00,  5.37item/s]


In [110]:
test_ds

<xarray.Dataset> Size: 127MB
Dimensions:                 (time: 12, lat: 23, long: 69, variable: 87,
                             months: 12, year_range: 3, model_name: 5,
                             ensemble_member: 3, month: 2)
Coordinates:
  * time                    (time) datetime64[ns] 96B 1980-01-01 ... 1980-12-01
  * lat                     (lat) float64 184B 50.87 50.87 50.86 ... 50.7 50.69
  * long                    (long) float64 552B 4.176 4.184 ... 4.734 4.742
  * variable                (variable) <U16 6kB 'ai' 'bio1' ... 'vpd_range'
  * months                  (months) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * year_range              (year_range) <U9 108B '2011-2040' ... '2071-2100'
  * model_name              (model_name) <U13 260B 'gfdl-esm4' ... 'ukesm1-0-ll'
  * ensemble_member         (ensemble_member) <U6 72B 'ssp126' 'ssp370' 'ssp585'
  * month                   (month) int64 16B 1 2
Data variables:
    chelsa_month            (variable, time, lat, long) float64 13MB nan ... nan
    chelsa_clim_ref_period  (variable, lat, long) float64 1MB 1.18 ... 5.568e+03
    chelsa_clim_ref_month   (variable, months, lat, long) float64 13MB nan .....
    chelsa_clim_sim_period  (variable, year_range, model_name, ensemble_member, lat, long) float64 50MB ...
    chelsa_clim_sim_month   (variable, year_range, month, model_name, ensemble_member, lat, long) float32 50MB ...

In [14]:
test_layer_m = test_chelsa_cube.generate_chelsa_month_layer("prototype_param.yaml")

-----Retrieving monthly CHELSA data for variable 'clt'-----


Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  5.35item/s]


-----Retrieving monthly CHELSA data for variable 'cmi'-----


Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  5.59item/s]


-----Retrieving monthly CHELSA data for variable 'hurs'-----


...

Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  4.90item/s]


-----Retrieving monthly CHELSA data for variable 'tasmin'-----


Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.73item/s]


-----Retrieving monthly CHELSA data for variable 'vpd'-----


Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  4.93item/s]


In [9]:
test_layer_m

<xarray.DataArray (variable: 11, time: 12, lat: 23, long: 69)> Size: 2MB
array([[[[5151.99999941, 5151.99999941, 5152.6666662 , ...,
          5211.99994806, 5211.99994806, 5211.99994806],
         [5151.99999941, 5151.99999941, 5152.6666662 , ...,
          5211.99994806, 5211.99994806, 5211.99994806],
         [5150.33333195, 5150.33333195, 5150.77777635, ...,
          5213.22218247, 5213.22218247, 5213.22218247],
         ...,
         [5168.66666046, 5168.66666046, 5162.77777556, ...,
          5268.11163722, 5268.11163722, 5268.11163722],
         [5168.33332746, 5168.33332746, 5163.22222095, ...,
          5274.8893972 , 5274.8893972 , 5274.8893972 ],
         [5168.33332746, 5168.33332746, 5163.22222095, ...,
          5274.8893972 , 5274.8893972 , 5274.8893972 ]],

        [[6263.99999584, 6263.99999584, 6265.33332997, ...,
          6208.66651341, 6208.66651341, 6208.66651341],
         [6263.99999584, 6263.99999584, 6265.33332997, ...,
          6208.66651341, 6208.66651341, 6208.66651341],
         [6255.66665947, 6255.66665947, 6257.44443839, ...,
          6201.88879732, 6201.88879732, 6201.88879732],
...
          2763.        , 2748.        , 2716.        ],
         [2872.        , 2875.        , 2906.        , ...,
          2752.        , 2742.        , 2712.        ],
         [2903.        , 2910.        , 2910.        , ...,
          2751.        , 2737.        , 2704.        ]],

        [[2304.        , 2303.        , 2299.        , ...,
          2286.        , 2269.        , 2267.        ],
         [2306.        , 2304.        , 2286.        , ...,
          2274.        , 2270.        , 2266.        ],
         [2280.        , 2280.        , 2280.        , ...,
          2246.        , 2242.        , 2238.        ],
         ...,
         [2202.        , 2216.        , 2215.        , ...,
          2108.        , 2099.        , 2088.        ],
         [2211.        , 2214.        , 2226.        , ...,
          2099.        , 2090.        , 2080.        ],
         [2206.        , 2222.        , 2209.        , ...,
          2085.        , 2079.        , 2072.        ]]]],
      shape=(11, 12, 23, 69))
Coordinates:
  * time      (time) datetime64[ns] 96B 1980-01-01 1980-02-01 ... 1980-12-01
  * lat       (lat) float64 184B 50.87 50.87 50.86 50.85 ... 50.71 50.7 50.69
  * long      (long) float64 552B 4.176 4.184 4.192 4.201 ... 4.726 4.734 4.742
  * variable  (variable) <U7 308B 'clt' 'cmi' 'hurs' ... 'tasmax' 'tasmin' 'vpd'

In [5]:
test_layer_ref_p = test_chelsa_cube.generate_chelsa_ref_period_layer("prototype_param.yaml")

-----Retrieving Reference Climatology CHELSA data for variable 'ai' in reference period 1981-2010-----
Complete
-----Retrieving Reference Climatology CHELSA data for variable 'bio10' in reference period 1981-2010-----
Complete
-----Retrieving Reference Climatology CHELSA data for variable 'bio11' in reference period 1981-2010-----
Complete
-----Retrieving Reference Climatology CHELSA data for variable 'bio12' in reference period 1981-2010-----
Complete
-----Retrieving Reference Climatology CHELSA data for variable 'bio13' in reference period 1981-2010-----
Complete
-----Retrieving Reference Climatology CHELSA data for variable 'bio14' in reference period 1981-2010-----
Complete
-----Retrieving Reference Climatology CHELSA data for variable 'bio15' in reference period 1981-2010-----
Complete
-----Retrieving Reference Climatology CHELSA data for variable 'bio16' in reference period 1981-2010-----
Complete
-----Retrieving Reference Climatology CHELSA data for variable 'bio17' in reference

In [10]:
test_layer_ref_p

<xarray.DataArray (variable: 76, lat: 23, long: 69)> Size: 965kB
array([[[1.18015158e+00, 1.18255329e+00, 1.18540442e+00, ...,
         1.13700640e+00, 1.13968790e+00, 1.14541054e+00],
        [1.18246496e+00, 1.18452954e+00, 1.18502748e+00, ...,
         1.14463651e+00, 1.14395237e+00, 1.14583743e+00],
        [1.18958151e+00, 1.18728614e+00, 1.18784654e+00, ...,
         1.15806282e+00, 1.15893531e+00, 1.16048992e+00],
        ...,
        [1.17370260e+00, 1.16899788e+00, 1.16737151e+00, ...,
         1.18518853e+00, 1.19208217e+00, 1.20514143e+00],
        [1.16658247e+00, 1.16406047e+00, 1.15795887e+00, ...,
         1.19367743e+00, 1.19901192e+00, 1.20935237e+00],
        [1.15844524e+00, 1.15460443e+00, 1.15378034e+00, ...,
         1.19719100e+00, 1.20685768e+00, 1.21573091e+00]],

       [[2.90600000e+03, 2.90500000e+03, 2.90500000e+03, ...,
         2.90600000e+03, 2.90600000e+03, 2.90400000e+03],
        [2.90500000e+03, 2.90500000e+03, 2.90500000e+03, ...,
         2.90600000e+03, 2.90600000e+03, 2.90600000e+03],
        [2.90500000e+03, 2.90500000e+03, 2.90500000e+03, ...,
         2.90500000e+03, 2.90500000e+03, 2.90600000e+03],
...
         2.43100000e+03, 2.41900000e+03, 2.40500000e+03],
        [2.50500000e+03, 2.50700000e+03, 2.51200000e+03, ...,
         2.42000000e+03, 2.40800000e+03, 2.39700000e+03],
        [2.51000000e+03, 2.51600000e+03, 2.51600000e+03, ...,
         2.41100000e+03, 2.39800000e+03, 2.38700000e+03]],

       [[5.80300000e+03, 5.79800000e+03, 5.79700000e+03, ...,
         5.93600000e+03, 5.90700000e+03, 5.84700000e+03],
        [5.74700000e+03, 5.76400000e+03, 5.76400000e+03, ...,
         5.93000000e+03, 5.93400000e+03, 5.92800000e+03],
        [5.74300000e+03, 5.74400000e+03, 5.74800000e+03, ...,
         5.87900000e+03, 5.87100000e+03, 5.88600000e+03],
        ...,
        [5.66600000e+03, 5.70300000e+03, 5.72400000e+03, ...,
         5.61500000e+03, 5.59000000e+03, 5.57300000e+03],
        [5.73100000e+03, 5.73600000e+03, 5.78100000e+03, ...,
         5.59600000e+03, 5.58700000e+03, 5.57100000e+03],
        [5.76500000e+03, 5.78200000e+03, 5.77500000e+03, ...,
         5.62200000e+03, 5.58500000e+03, 5.56800000e+03]]],
      shape=(76, 23, 69))
Coordinates:
  * lat       (lat) float64 184B 50.87 50.87 50.86 50.85 ... 50.71 50.7 50.69
  * long      (long) float64 552B 4.176 4.184 4.192 4.201 ... 4.726 4.734 4.742
  * variable  (variable) <U16 5kB 'ai' 'bio10' 'bio11' ... 'vpd_min' 'vpd_range'
Attributes:
    year_range:  1981-2010

In [6]:
test_layer_ref_m = test_chelsa_cube.generate_chelsa_ref_month_layer("prototype_param.yaml")

-----Retrieving monthly Reference Climatology CHELSA data for variable 'clt' in reference period 1981-2010-----


Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:04<00:00,  2.92item/s]


-----Retrieving monthly Reference Climatology CHELSA data for variable 'cmi' in reference period 1981-2010-----


Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:04<00:00,  2.76item/s]


-----Retrieving monthly Reference Climatology CHELSA data for variable 'hurs' in reference period 1981-2010-----


...

Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:04<00:00,  2.61item/s]


-----Retrieving monthly Reference Climatology CHELSA data for variable 'tasmin' in reference period 1981-2010-----


Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:05<00:00,  2.33item/s]


-----Retrieving monthly Reference Climatology CHELSA data for variable 'vpd' in reference period 1981-2010-----


Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:04<00:00,  2.84item/s]


In [11]:
test_layer_ref_m

<xarray.DataArray (variable: 11, months: 12, lat: 23, long: 69)> Size: 2MB
array([[[[5172.00000634, 5172.00000634, 5174.3333396 , ...,
          5185.00000152, 5185.00000152, 5185.00000152],
         [5172.00000634, 5172.00000634, 5174.3333396 , ...,
          5185.00000152, 5185.00000152, 5185.00000152],
         [5180.33334198, 5180.33334198, 5181.00000726, ...,
          5187.55555223, 5187.55555223, 5187.55555223],
         ...,
         [5198.66663723, 5198.66663723, 5194.66664918, ...,
          5290.66730964, 5290.66730964, 5290.66730964],
         [5195.33330357, 5195.33330357, 5192.9999835 , ...,
          5300.6673136 , 5300.6673136 , 5300.6673136 ],
         [5195.33330357, 5195.33330357, 5192.9999835 , ...,
          5300.6673136 , 5300.6673136 , 5300.6673136 ]],

        [[4914.00000356, 4914.00000356, 4915.0000033 , ...,
          4913.66670415, 4913.66670415, 4913.66670415],
         [4914.00000356, 4914.00000356, 4915.0000033 , ...,
          4913.66670415, 4913.66670415, 4913.66670415],
         [4919.00000488, 4919.00000488, 4918.88889263, ...,
          4917.66668818, 4917.66668818, 4917.66668818],
...
          3100.        , 3082.        , 3066.        ],
         [3199.        , 3200.        , 3213.        , ...,
          3084.        , 3072.        , 3058.        ],
         [3210.        , 3220.        , 3216.        , ...,
          3080.        , 3062.        , 3047.        ]],

        [[2546.        , 2542.        , 2539.        , ...,
          2549.        , 2545.        , 2534.        ],
         [2539.        , 2540.        , 2538.        , ...,
          2541.        , 2540.        , 2537.        ],
         [2529.        , 2530.        , 2529.        , ...,
          2525.        , 2522.        , 2523.        ],
         ...,
         [2493.        , 2499.        , 2505.        , ...,
          2431.        , 2419.        , 2405.        ],
         [2505.        , 2507.        , 2512.        , ...,
          2420.        , 2408.        , 2397.        ],
         [2510.        , 2516.        , 2516.        , ...,
          2411.        , 2398.        , 2387.        ]]]],
      shape=(11, 12, 23, 69))
Coordinates:
  * months    (months) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * lat       (lat) float64 184B 50.87 50.87 50.86 50.85 ... 50.71 50.7 50.69
  * long      (long) float64 552B 4.176 4.184 4.192 4.201 ... 4.726 4.734 4.742
  * variable  (variable) <U7 308B 'clt' 'cmi' 'hurs' ... 'tasmax' 'tasmin' 'vpd'
Attributes:
    year_range:  1981-2010

In [7]:
test_layer_sim_p = test_chelsa_cube.generate_chelsa_sim_period_layer("prototype_param.yaml")

-----Retrieving Simulation (period) CHELSA data for variable 'bio10'-----


Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:16<00:00,  2.67item/s]


-----Retrieving Simulation (period) CHELSA data for variable 'bio11'-----


Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:15<00:00,  2.87item/s]


-----Retrieving Simulation (period) CHELSA data for variable 'bio12'-----


...

Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:17<00:00,  2.53item/s]


-----Retrieving Simulation (period) CHELSA data for variable 'scd'-----


Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:15<00:00,  2.82item/s]


-----Retrieving Simulation (period) CHELSA data for variable 'swe'-----


Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:16<00:00,  2.74item/s]


In [12]:
test_layer_sim_p

<xarray.DataArray (variable: 46, year_range: 3, model_name: 5,
                   ensemble_member: 3, lat: 23, long: 69)> Size: 26MB
array([[[[[[ 2920.,  2919.,  2919., ...,  2920.,  2920.,  2918.],
           [ 2919.,  2919.,  2919., ...,  2920.,  2920.,  2920.],
           [ 2919.,  2919.,  2919., ...,  2919.,  2919.,  2920.],
           ...,
           [ 2917.,  2918.,  2918., ...,  2915.,  2915.,  2915.],
           [ 2918.,  2918.,  2919., ...,  2915.,  2915.,  2915.],
           [ 2919.,  2919.,  2919., ...,  2916.,  2915.,  2915.]],

          [[ 2920.,  2920.,  2920., ...,  2920.,  2920.,  2918.],
           [ 2919.,  2919.,  2919., ...,  2920.,  2920.,  2920.],
           [ 2919.,  2919.,  2919., ...,  2919.,  2919.,  2920.],
           ...,
           [ 2917.,  2918.,  2918., ...,  2916.,  2916.,  2916.],
           [ 2919.,  2919.,  2920., ...,  2916.,  2916.,  2916.],
           [ 2919.,  2920.,  2920., ...,  2917.,  2916.,  2916.]],

          [[ 2915.,  2915.,  2915., ...,  2915.,  2915.,  2913.],
           [ 2914.,  2914.,  2914., ...,  2915.,  2915.,  2915.],
           [ 2914.,  2914.,  2914., ...,  2914.,  2914.,  2915.],
           ...,
...
           [65535., 65535., 65535., ..., 65535., 65535., 65535.],
           [65535., 65535., 65535., ..., 65535., 65535., 65535.],
           [65535., 65535., 65535., ..., 65535., 65535., 65535.]],

          [[65535., 65535., 65535., ..., 65535., 65535., 65535.],
           [65535., 65535., 65535., ..., 65535., 65535., 65535.],
           [65535., 65535., 65535., ..., 65535., 65535., 65535.],
           ...,
           [65535., 65535., 65535., ..., 65535., 65535., 65535.],
           [65535., 65535., 65535., ..., 65535., 65535., 65535.],
           [65535., 65535., 65535., ..., 65535., 65535., 65535.]],

          [[65535., 65535., 65535., ..., 65535., 65535., 65535.],
           [65535., 65535., 65535., ..., 65535., 65535., 65535.],
           [65535., 65535., 65535., ..., 65535., 65535., 65535.],
           ...,
           [65535., 65535., 65535., ..., 65535., 65535., 65535.],
           [65535., 65535., 65535., ..., 65535., 65535., 65535.],
           [65535., 65535., 65535., ..., 65535., 65535., 65535.]]]]]],
      shape=(46, 3, 5, 3, 23, 69))
Coordinates:
  * year_range       (year_range) <U9 108B '2011-2040' '2041-2070' '2071-2100'
  * model_name       (model_name) <U13 260B 'gfdl-esm4' ... 'ukesm1-0-ll'
  * ensemble_member  (ensemble_member) <U6 72B 'ssp126' 'ssp370' 'ssp585'
  * lat              (lat) float64 184B 50.87 50.87 50.86 ... 50.71 50.7 50.69
  * long             (long) float64 552B 4.176 4.184 4.192 ... 4.726 4.734 4.742
  * variable         (variable) <U8 1kB 'bio10' 'bio11' 'bio12' ... 'scd' 'swe'

In [8]:
test_sim_m = test_chelsa_cube.generate_chelsa_sim_month_layer("prototype_param.yaml")

-----Retrieving Simulation (monthly) CHELSA data for variable 'pr'-----


Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:41<00:00,  2.16item/s]


-----Retrieving Simulation (monthly) CHELSA data for variable 'tas'-----


Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:40<00:00,  2.22item/s]


-----Retrieving Simulation (monthly) CHELSA data for variable 'tasmax'-----


Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:41<00:00,  2.15item/s]


-----Retrieving Simulation (monthly) CHELSA data for variable 'tasmin'-----


Processing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [00:43<00:00,  2.06item/s]


In [13]:
test_sim_m

<xarray.DataArray (variable: 4, year_range: 3, month: 2, model_name: 5,
                   ensemble_member: 3, lat: 23, long: 69)> Size: 1MB
array([[[[[[[ 685,  686,  688, ...,  686,  686,  686],
            [ 684,  685,  687, ...,  691,  690,  690],
            [ 688,  688,  689, ...,  697,  696,  696],
            ...,
            [ 681,  681,  680, ...,  697,  702,  707],
            [ 679,  678,  677, ...,  702,  705,  710],
            [ 677,  676,  675, ...,  707,  710,  713]],

           [[ 706,  708,  710, ...,  713,  713,  714],
            [ 706,  707,  709, ...,  719,  718,  718],
            [ 711,  711,  712, ...,  726,  725,  725],
            ...,
            [ 707,  708,  707, ...,  729,  734,  739],
            [ 705,  705,  704, ...,  734,  738,  743],
            [ 703,  703,  702, ...,  740,  743,  746]],

           [[ 727,  729,  731, ...,  736,  736,  736],
            [ 727,  728,  730, ...,  742,  741,  741],
            [ 732,  732,  733, ...,  749,  748,  748],
            ...,
...
            [2758, 2759, 2759, ..., 2755, 2755, 2755],
            [2759, 2759, 2760, ..., 2755, 2755, 2755],
            [2760, 2760, 2760, ..., 2756, 2755, 2755]],

           [[2781, 2781, 2781, ..., 2780, 2780, 2779],
            [2781, 2781, 2781, ..., 2780, 2780, 2780],
            [2780, 2780, 2780, ..., 2780, 2779, 2780],
            ...,
            [2778, 2779, 2779, ..., 2776, 2776, 2776],
            [2779, 2779, 2780, ..., 2776, 2776, 2776],
            [2780, 2780, 2780, ..., 2777, 2776, 2776]],

           [[2789, 2789, 2789, ..., 2788, 2788, 2787],
            [2789, 2789, 2789, ..., 2788, 2788, 2788],
            [2788, 2788, 2788, ..., 2788, 2787, 2788],
            ...,
            [2786, 2787, 2787, ..., 2784, 2784, 2784],
            [2787, 2787, 2788, ..., 2784, 2784, 2784],
            [2788, 2788, 2788, ..., 2785, 2784, 2784]]]]]]],
      shape=(4, 3, 2, 5, 3, 23, 69), dtype=uint16)
Coordinates:
  * year_range       (year_range) <U9 108B '2011-2040' '2041-2070' '2071-2100'
  * month            (month) int64 16B 1 2
  * model_name       (model_name) <U13 260B 'gfdl-esm4' ... 'ukesm1-0-ll'
  * ensemble_member  (ensemble_member) <U6 72B 'ssp126' 'ssp370' 'ssp585'
  * lat              (lat) float64 184B 50.87 50.87 50.86 ... 50.71 50.7 50.69
  * long             (long) float64 552B 4.176 4.184 4.192 ... 4.726 4.734 4.742
  * variable         (variable) <U6 96B 'pr' 'tas' 'tasmax' 'tasmin'

In [112]:
test_ds = xr.Dataset({"chelsa_month":test_layer_m, "chelsa_ref_period":test_layer_ref_p})

In [113]:
test_ds

<xarray.Dataset> Size: 14MB
Dimensions:            (time: 12, lat: 23, long: 69, variable: 87)
Coordinates:
  * time               (time) datetime64[ns] 96B 1980-01-01 ... 1980-12-01
  * lat                (lat) float64 184B 50.87 50.87 50.86 ... 50.71 50.7 50.69
  * long               (long) float64 552B 4.176 4.184 4.192 ... 4.734 4.742
  * variable           (variable) <U16 6kB 'ai' 'bio1' ... 'vpd_min' 'vpd_range'
Data variables:
    chelsa_month       (variable, time, lat, long) float64 13MB nan nan ... nan
    chelsa_ref_period  (variable, lat, long) float64 1MB 1.18 ... 5.568e+03

In [114]:
test_ds = xr.Dataset({})
test_ds["chelsa_month"]=test_layer_m
test_ds["chelsa_ref_period"]=test_layer_ref_p

In [115]:
test_ds

<xarray.Dataset> Size: 2MB
Dimensions:            (time: 12, lat: 23, long: 69, variable: 11)
Coordinates:
  * time               (time) datetime64[ns] 96B 1980-01-01 ... 1980-12-01
  * lat                (lat) float64 184B 50.87 50.87 50.86 ... 50.71 50.7 50.69
  * long               (long) float64 552B 4.176 4.184 4.192 ... 4.734 4.742
  * variable           (variable) <U7 308B 'clt' 'cmi' 'hurs' ... 'tasmin' 'vpd'
Data variables:
    chelsa_month       (variable, time, lat, long) float64 2MB 5.152e+03 ... ...
    chelsa_ref_period  (variable, lat, long) float64 140kB nan nan ... nan nan

In [2]:
import xarray as xr
import numpy as np

In [8]:
test_da_1 = xr.DataArray(data=np.random.uniform(0,1, (10,10)), dims=("lat", "long"), coords={"lat":np.arange(0,10), "long":np.arange(0,10)})
test_da_2 = xr.DataArray(data=np.random.uniform(0,1, (10,10)), dims=("lat", "long"), coords={"lat":np.arange(0,10), "long":np.arange(0,10)})
test_da_3 = xr.DataArray(data=np.random.uniform(0,1, (10,10)), dims=("lat", "long"), coords={"lat":np.arange(0,10), "long":np.arange(0,10)})

In [9]:
test_da_1

<xarray.DataArray (lat: 10, long: 10)> Size: 800B
array([[0.02959257, 0.45593101, 0.16076704, 0.9186148 , 0.71839676,
        0.58015552, 0.20364439, 0.20853443, 0.31825696, 0.32120504],
       [0.23824148, 0.59299175, 0.33761381, 0.59602328, 0.30617284,
        0.58182474, 0.66648707, 0.56573044, 0.9446082 , 0.72012934],
       [0.58539838, 0.73876833, 0.27260168, 0.45748952, 0.43403108,
        0.46100631, 0.92948863, 0.8648683 , 0.99755832, 0.91064708],
       [0.47366164, 0.96729498, 0.42184181, 0.34110248, 0.86017503,
        0.18233633, 0.19996729, 0.37685195, 0.28809682, 0.99102054],
       [0.2901896 , 0.37081097, 0.48285048, 0.77553617, 0.78415132,
        0.34580269, 0.58887864, 0.36646321, 0.38209558, 0.81135832],
       [0.28489477, 0.95273136, 0.33787758, 0.88127769, 0.79485919,
        0.20511674, 0.61473181, 0.99392511, 0.6499825 , 0.63998769],
       [0.10518263, 0.86079739, 0.27342838, 0.20962898, 0.78261359,
        0.9682181 , 0.16942663, 0.80728967, 0.20778417, 0.02816755],
       [0.22205923, 0.53338708, 0.11894127, 0.09383925, 0.67961745,
        0.36820496, 0.96556395, 0.19489972, 0.23928707, 0.87338511],
       [0.80842236, 0.71860566, 0.78321708, 0.08418115, 0.28022151,
        0.61660172, 0.03514246, 0.59082474, 0.86681532, 0.25398329],
       [0.52491158, 0.35628441, 0.75627323, 0.73749815, 0.77532822,
        0.9497408 , 0.50783497, 0.10692085, 0.8572524 , 0.3145847 ]])
Coordinates:
  * lat      (lat) int64 80B 0 1 2 3 4 5 6 7 8 9
  * long     (long) int64 80B 0 1 2 3 4 5 6 7 8 9

In [13]:
# Wrap each DataArray into a Dataset
test_dict = {
    "static": {
        "t1": test_da_1.to_dataset(name="t1"),
        "t2": test_da_2.to_dataset(name="t2")
    },
    "dynamic": {
        "month": {
            "t3": test_da_3.to_dataset(name="t3")
        }
    }
}

In [14]:
test_dict

{'static': {'t1': <xarray.Dataset> Size: 960B
  Dimensions:  (lat: 10, long: 10)
  Coordinates:
    * lat      (lat) int64 80B 0 1 2 3 4 5 6 7 8 9
    * long     (long) int64 80B 0 1 2 3 4 5 6 7 8 9
  Data variables:
      t1       (lat, long) float64 800B 0.02959 0.4559 0.1608 ... 0.8573 0.3146,
  't2': <xarray.Dataset> Size: 960B
  Dimensions:  (lat: 10, long: 10)
  Coordinates:
    * lat      (lat) int64 80B 0 1 2 3 4 5 6 7 8 9
    * long     (long) int64 80B 0 1 2 3 4 5 6 7 8 9
  Data variables:
      t2       (lat, long) float64 800B 0.09535 0.2767 0.9736 ... 0.3828 0.3243},
 'dynamic': {'month': {'t3': <xarray.Dataset> Size: 960B
   Dimensions:  (lat: 10, long: 10)
   Coordinates:
     * lat      (lat) int64 80B 0 1 2 3 4 5 6 7 8 9
     * long     (long) int64 80B 0 1 2 3 4 5 6 7 8 9
   Data variables:
       t3       (lat, long) float64 800B 0.8096 0.6128 0.7187 ... 0.5734 0.6677}}}

In [16]:
test_dt = xr.DataTree.from_dict(test_dict)

TypeError: invalid values: {'t1': <xarray.Dataset> Size: 960B
Dimensions:  (lat: 10, long: 10)
Coordinates:
  * lat      (lat) int64 80B 0 1 2 3 4 5 6 7 8 9
  * long     (long) int64 80B 0 1 2 3 4 5 6 7 8 9
Data variables:
    t1       (lat, long) float64 800B 0.02959 0.4559 0.1608 ... 0.8573 0.3146, 't2': <xarray.Dataset> Size: 960B
Dimensions:  (lat: 10, long: 10)
Coordinates:
  * lat      (lat) int64 80B 0 1 2 3 4 5 6 7 8 9
  * long     (long) int64 80B 0 1 2 3 4 5 6 7 8 9
Data variables:
    t2       (lat, long) float64 800B 0.09535 0.2767 0.9736 ... 0.3828 0.3243}

In [3]:
import xarray as xr
import numpy as np
from datatree import DataTree

# Create test DataArrays
test_da_1 = xr.DataArray(
    data=np.random.uniform(0, 1, (10, 10)),
    dims=("lat", "long"),
    coords={"lat": np.arange(10), "long": np.arange(10)}
)
test_da_2 = test_da_1.copy(data=np.random.uniform(0, 1, (10, 10)))
test_da_3 = test_da_1.copy(data=np.random.uniform(0, 1, (10, 10)))

# Convert to Datasets with unique variable names
ds1 = test_da_1.to_dataset(name="data")
ds2 = test_da_2.to_dataset(name="data")
ds3 = test_da_3.to_dataset(name="data")  # ✅ Convert to Dataset

# Build dictionary with DataTree-compatible structure
test_dict = {
    "static": {
        "t1": ds1,
        "t2": ds2
    },
    "dynamic": {
        "t3": ds3  # ✅ Now it's a Dataset
    }
}

# Create DataTree
test_dt = DataTree.from_dict(test_dict)

# Display structure
print(test_dt)


TypeError: data object is not an xarray Dataset, DataArray, or None, it is of type <class 'dict'>

In [11]:
test_da_11 = xr.DataArray(
    data=np.random.uniform(0, 1, (10, 10)),
    dims=("lat", "long"),
    coords={"lat": np.arange(10), "long": np.arange(10)}
)
test_da_12 = xr.DataArray(
    data=np.random.uniform(0, 1, (10, 10)),
    dims=("lat", "long"),
    coords={"lat": np.arange(10), "long": np.arange(10)}
)
test_da_13 = xr.DataArray(
    data=np.random.uniform(0, 1, (10, 10)),
    dims=("lat", "long"),
    coords={"lat": np.arange(10), "long": np.arange(10)}
)
names=["var1", "var2", "var3"]
test_ds_1 = xr.Dataset(dict(zip(names, [test_da_11, test_da_12, test_da_13])))

In [12]:
test_ds_1

<xarray.Dataset> Size: 3kB
Dimensions:  (lat: 10, long: 10)
Coordinates:
  * lat      (lat) int64 80B 0 1 2 3 4 5 6 7 8 9
  * long     (long) int64 80B 0 1 2 3 4 5 6 7 8 9
Data variables:
    var1     (lat, long) float64 800B 0.03799 0.6316 0.432 ... 0.9184 0.1057
    var2     (lat, long) float64 800B 0.08239 0.9878 0.1158 ... 0.4942 0.9805
    var3     (lat, long) float64 800B 0.06505 0.0995 0.8211 ... 0.8238 0.6393

In [15]:
test_da_2 = xr.DataArray(data=np.random.uniform(0,1, (3,10,10)), dims=("time","lat", "long"), coords={"time":[1,2,3], "lat":np.arange(0,10), "long":np.arange(0,10)})
test_da_3 = xr.DataArray(data=np.random.uniform(0,1, (10,10)), dims=("lat", "long"), coords={"lat":np.arange(0,10), "long":np.arange(0,10)})
# Convert to Datasets with unique variable names
ds2 = test_da_2.to_dataset(name="data")
ds3 = test_da_3.to_dataset(name="data")
ds2

<xarray.Dataset> Size: 3kB
Dimensions:  (time: 3, lat: 10, long: 10)
Coordinates:
  * time     (time) int64 24B 1 2 3
  * lat      (lat) int64 80B 0 1 2 3 4 5 6 7 8 9
  * long     (long) int64 80B 0 1 2 3 4 5 6 7 8 9
Data variables:
    data     (time, lat, long) float64 2kB 0.484 0.2507 0.6946 ... 0.5336 0.9525

In [5]:

test_da_2 = xr.DataArray(data=np.random.uniform(0,1, (3,10,10)), dims=("time","lat", "long"), coords={"time":[1,2,3], "lat":np.arange(0,10), "long":np.arange(0,10)})
test_da_3 = xr.DataArray(data=np.random.uniform(0,1, (10,10)), dims=("lat", "long"), coords={"lat":np.arange(0,10), "long":np.arange(0,10)})
# Convert to Datasets with unique variable names
ds2 = test_da_2.to_dataset(name="data")
ds3 = test_da_3.to_dataset(name="data")  
# Leaf wrappers
t1 = DataTree(name="t1", data=ds1)
t2 = DataTree(name="t2", data=ds2)
t3 = DataTree(name="t3", data=ds3)

# Intermediate branches
static  = DataTree(name="static",  children={"t1": t1, "t2": t2})
dynamic = DataTree(name="dynamic", children={"t3": t3})

# Root assembly
root = DataTree(name="root", children={"static": static, "dynamic": dynamic})

print(root)


DataTree('root', parent=None)
├── DataTree('static')
│   ├── DataTree('t1')
│   │       Dimensions:  (lat: 10, long: 10)
│   │       Coordinates:
│   │         * lat      (lat) int64 80B 0 1 2 3 4 5 6 7 8 9
│   │         * long     (long) int64 80B 0 1 2 3 4 5 6 7 8 9
│   │       Data variables:
│   │           data     (lat, long) float64 800B 0.3949 0.8842 0.8779 ... 0.5814 0.3441
│   └── DataTree('t2')
│           Dimensions:  (lat: 10, long: 10)
│           Coordinates:
│             * lat      (lat) int64 80B 0 1 2 3 4 5 6 7 8 9
│             * long     (long) int64 80B 0 1 2 3 4 5 6 7 8 9
│           Data variables:
│               data     (lat, long) float64 800B 0.3311 0.834 0.7325 ... 0.05925 0.5339
└── DataTree('dynamic')
    └── DataTree('t3')
            Dimensions:  (lat: 10, long: 10)
            Coordinates:
              * lat      (lat) int64 80B 0 1 2 3 4 5 6 7 8 9
              * long     (long) int64 80B 0 1 2 3 4 5 6 7 8 9
            Data variables:
             

In [9]:
test_ds_2 = test_da_1.copy(data=np.random.uniform(0, 1, (10, 10)))

In [10]:
test_ds_2

<xarray.DataArray (lat: 10, long: 10)> Size: 800B
array([[0.89265379, 0.02486652, 0.38680787, 0.04484291, 0.1267423 ,
        0.77495508, 0.54137767, 0.72090411, 0.92528234, 0.92291488],
       [0.07943498, 0.79220975, 0.4883737 , 0.45676193, 0.82237889,
        0.61632246, 0.48345825, 0.59772699, 0.90112918, 0.52436007],
       [0.83208176, 0.06505952, 0.1880781 , 0.9955452 , 0.99947958,
        0.14943266, 0.16848099, 0.57761496, 0.90577271, 0.24193241],
       [0.9517107 , 0.82003573, 0.98733455, 0.28528329, 0.32691248,
        0.09245325, 0.93501904, 0.22857379, 0.44349252, 0.81280853],
       [0.38859915, 0.59747896, 0.81538323, 0.53224009, 0.92886543,
        0.78442608, 0.82706866, 0.25606408, 0.19988322, 0.74341964],
       [0.47750823, 0.22645243, 0.39895754, 0.09046761, 0.16190382,
        0.91949051, 0.03874578, 0.70330138, 0.29323217, 0.69874532],
       [0.96882014, 0.51457061, 0.14990428, 0.71919182, 0.63393036,
        0.68960077, 0.43051508, 0.21079477, 0.76918283, 0.60712525],
       [0.00578027, 0.75446531, 0.58786439, 0.86583861, 0.56476834,
        0.78080405, 0.91205237, 0.67479077, 0.19399705, 0.53956336],
       [0.74988592, 0.34500171, 0.89114711, 0.9885592 , 0.32308302,
        0.52393299, 0.38671986, 0.8461274 , 0.67644887, 0.52006998],
       [0.21256897, 0.96088206, 0.3477166 , 0.39517439, 0.66528104,
        0.66159652, 0.19254014, 0.35760333, 0.87534392, 0.06104605]])
Coordinates:
  * lat      (lat) int64 80B 0 1 2 3 4 5 6 7 8 9
  * long     (long) int64 80B 0 1 2 3 4 5 6 7 8 9